# Building a Conversational Chatbot

This notebook demonstrates how to create a simple but functional chatbot using Hugging Face's Transformers library. We'll start by installing the necessary dependencies.

In [1]:
# Install required libraries
!pip install -q transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 5.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12

## Import Dependencies

This cell imports all the necessary libraries for our conversational chatbot. We need PyTorch for the deep learning components, transformers for the language model, and various utilities for data handling and visualization.

In [2]:
# Import necessary libraries
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from datasets import load_dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import re
from IPython.display import display, HTML

In [5]:
# Determine the available computing device
compute_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {compute_device}")

Using device: cuda


## Load Model and Tokenizer

This cell loads the pretrained language model and its associated tokenizer. We're using a smaller model (BlenderBot) for demonstration purposes, but you could substitute a larger model for better performance if your hardware allows.

In [6]:
# Initialize the language model and tokenizer
selected_model = "facebook/blenderbot-400M-distill"
print(f"Loading model: {selected_model}")

# Load the components from Hugging Face
tokenizer = AutoTokenizer.from_pretrained(selected_model)
model = AutoModelForCausalLM.from_pretrained(selected_model).to(compute_device)

print("Model and tokenizer loaded successfully!")

Loading model: facebook/blenderbot-400M-distill
Model and tokenizer loaded successfully!


## Helper Functions for Chat Processing

This cell defines utility functions for formatting conversations and generating responses. These functions handle the conversation flow, including formatting messages based on their role (user or assistant) and using the model to generate appropriate responses.

In [7]:
# Define helper functions for conversation management

def create_message_format(speaker_role, message_text):
    """Format a message based on who is speaking"""
    if speaker_role == "user":
        return f"User: {message_text}"
    else:
        return f"Assistant: {message_text}"

def compile_conversation_history(message_list):
    """Convert the list of messages into a formatted conversation string"""
    return "\n".join([create_message_format(msg["role"], msg["content"]) for msg in message_list])

def create_bot_response(message_list, response_length=128):
    """Generate the assistant's response based on conversation context"""
    # Format the full conversation history
    formatted_history = compile_conversation_history(message_list)

    # Add the assistant prefix for the new response
    full_prompt = formatted_history + "\nAssistant:"

    # Convert to model input format
    encoded_input = tokenizer(full_prompt, return_tensors="pt").to(compute_device)

    # Generate response without calculating gradients
    with torch.no_grad():
        generated_ids = model.generate(
            encoded_input["input_ids"],
            max_length=encoded_input["input_ids"].shape[1] + response_length,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    # Convert token IDs back to text
    complete_output = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    # Extract only the assistant's new response
    bot_response = complete_output.split("Assistant:")[-1].strip()

    return bot_response

## Basic Chat Interface

This cell implements a simple text-based chat interface that allows users to interact with the chatbot in the console. It handles the conversation flow, maintains conversation history, and displays the bot's responses.

In [8]:
# Implement a simple console-based chat interface
conversation_history = []

def start_text_chat():
    """Start an interactive text-based chat with the bot"""
    print("\n===== Conversational Chatbot Demo =====")
    print("Type 'exit' to end the conversation\n")

    while True:
        # Get input from the user
        user_message = input("User: ")

        # Check if user wants to exit
        if user_message.lower() == "exit":
            print("\nThank you for chatting! Goodbye.")
            break

        # Store the user's message
        conversation_history.append({"role": "user", "content": user_message})

        # Indicate that the model is processing
        print("Assistant is thinking...")

        # Generate and get response
        bot_message = create_bot_response(conversation_history)

        # Store the bot's response
        conversation_history.append({"role": "assistant", "content": bot_message})

        # Display the response
        print(f"Assistant: {bot_message}\n")

## Enhanced Chatbot Features

This cell implements advanced features to make our chatbot more dynamic and contextually aware:
1. Persona customization - allows changing the bot's personality
2. Memory management - for clearing or summarizing conversations
3. Sentiment analysis - to detect the emotional tone of messages

In [9]:
# Implement enhanced chatbot features

# Feature 1: Bot persona customization
def configure_bot_personality(personality_description):
    """Configure the chatbot's personality"""
    # Check if there's already a system message
    if len(conversation_history) == 0 or conversation_history[0]["role"] != "system":
        # Add a new system message at the beginning
        conversation_history.insert(0, {"role": "system", "content": personality_description})
    else:
        # Update existing system message
        conversation_history[0] = {"role": "system", "content": personality_description}
    print(f"Bot personality configured as: {personality_description}")

# Feature 2: Conversation management
def reset_conversation():
    """Clear the entire conversation history"""
    conversation_history.clear()
    print("Conversation history has been reset.")

def create_conversation_summary():
    """Generate a brief summary of the current conversation"""
    # Check if there's enough conversation to summarize
    if len(conversation_history) <= 2:
        return "The conversation has just begun."

    # Create a prompt for summarization
    summary_prompt = "Summarize this conversation:\n\n" + compile_conversation_history(conversation_history)

    # Use the model to generate a summary
    encoded_input = tokenizer(summary_prompt, return_tensors="pt").to(compute_device)
    with torch.no_grad():
        output_ids = model.generate(
            encoded_input["input_ids"],
            max_length=encoded_input["input_ids"].shape[1] + 100,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    # Extract just the summary (removing the original prompt)
    full_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    summary_text = full_output.replace(summary_prompt, "").strip()
    return summary_text

# Feature 3: Basic sentiment detection
def detect_message_sentiment(message_text):
    """Perform simple keyword-based sentiment analysis"""
    # Define sentiment indicator words
    positive_indicators = ["good", "great", "happy", "positive", "excellent",
                          "wonderful", "love", "like", "enjoy"]
    negative_indicators = ["bad", "terrible", "sad", "negative", "awful",
                          "horrible", "hate", "dislike", "disappointing"]

    # Convert to lowercase for comparison
    message_text = message_text.lower()

    # Count occurrences of sentiment words
    positive_matches = sum(1 for word in positive_indicators if word in message_text)
    negative_matches = sum(1 for word in negative_indicators if word in message_text)

    # Determine overall sentiment
    if positive_matches > negative_matches:
        return "positive"
    elif negative_matches > positive_matches:
        return "negative"
    else:
        return "neutral"

## Interactive Widget-Based Interface

This cell creates a more user-friendly interface using iPython widgets. It provides a text input field, buttons for sending messages and clearing the chat, and a dropdown menu to select different bot personas. This interface is particularly useful in notebook environments like Colab.

In [10]:
# Create an interactive widget-based chat interface
from IPython.display import display, HTML
import ipywidgets as widgets

def launch_widget_interface():
    """Launch an interactive widget-based chat interface"""
    # Create the interface widgets
    chat_display = widgets.Output()
    message_input = widgets.Text(placeholder="Type your message here...")
    send_button = widgets.Button(description="Send")
    clear_button = widgets.Button(description="Clear Chat")

    # Create persona selection dropdown
    personality_selector = widgets.Dropdown(
        options=[
            'Helpful Assistant',
            'Travel Guide',
            'Tech Support',
            'Friendly Friend',
            'Professional Colleague'
        ],
        value='Helpful Assistant',
        description='Bot Persona:'
    )

    # Display the interface components
    display(HTML("<h3>Conversational Chatbot</h3>"))
    display(personality_selector)
    display(widgets.HBox([message_input, send_button, clear_button]))
    display(chat_display)

    # Initialize the chat
    chat_messages = []
    configure_bot_personality("You are a helpful, respectful and honest assistant.")

    # Define button handlers
    def handle_send_button(button):
        user_text = message_input.value
        if not user_text.strip():
            return

        # Clear input field
        message_input.value = ""

        # Add user message to conversation
        chat_messages.append({"role": "user", "content": user_text})

        with chat_display:
            print(f"User: {user_text}")
            print("Assistant is thinking...")

            # Generate the bot's response
            bot_text = create_bot_response(chat_messages)

            # Add bot message to conversation
            chat_messages.append({"role": "assistant", "content": bot_text})

            print(f"Assistant: {bot_text}\n")

    def handle_clear_button(button):
        with chat_display:
            chat_display.clear_output()
            chat_messages.clear()
            # Apply the current persona
            selected_persona = personality_selector.value
            if selected_persona == "Helpful Assistant":
                configure_bot_personality("You are a helpful, respectful and honest assistant.")
            elif selected_persona == "Travel Guide":
                configure_bot_personality("You are a knowledgeable travel guide who provides detailed information about destinations, travel tips, and local customs.")
            elif selected_persona == "Tech Support":
                configure_bot_personality("You are a patient technical support specialist who helps users troubleshoot their computer and software issues.")
            elif selected_persona == "Friendly Friend":
                configure_bot_personality("You are a friendly and supportive friend who offers empathy, advice, and casual conversation.")
            elif selected_persona == "Professional Colleague":
                configure_bot_personality("You are a professional colleague who communicates in a business-appropriate manner, focusing on tasks and efficiency.")
            print("Chat has been cleared. You can start a new conversation.")

    def handle_persona_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            with chat_display:
                if change['new'] == "Helpful Assistant":
                    configure_bot_personality("You are a helpful, respectful and honest assistant.")
                elif change['new'] == "Travel Guide":
                    configure_bot_personality("You are a knowledgeable travel guide who provides detailed information about destinations, travel tips, and local customs.")
                elif change['new'] == "Tech Support":
                    configure_bot_personality("You are a patient technical support specialist who helps users troubleshoot their computer and software issues.")
                elif change['new'] == "Friendly Friend":
                    configure_bot_personality("You are a friendly and supportive friend who offers empathy, advice, and casual conversation.")
                elif change['new'] == "Professional Colleague":
                    configure_bot_personality("You are a professional colleague who communicates in a business-appropriate manner, focusing on tasks and efficiency.")
                print(f"Bot persona changed to: {change['new']}")

    # Connect the event handlers
    send_button.on_click(handle_send_button)
    clear_button.on_click(handle_clear_button)
    personality_selector.observe(handle_persona_change)

    # Enable sending messages with Enter key
    def handle_keypress(change):
        if change['type'] == 'change' and change['name'] == 'value':
            if change['new'].endswith('\n'):
                message_input.value = change['new'].rstrip('\n')
                handle_send_button(None)

    message_input.observe(handle_keypress)

## Evaluation Functions

This cell implements functions for evaluating the quality of the chatbot's responses. It includes:
1. A response quality evaluator with multiple criteria
2. A test suite with pre-defined conversation scenarios

These tools help assess the bot's performance across different types of interactions.

In [11]:
# Functions for evaluating the chatbot's performance

def assess_response_quality(response_text, evaluation_criteria=None):
    """Evaluate a bot response based on multiple quality criteria"""
    if evaluation_criteria is None:
        evaluation_criteria = {
            "relevance": "Is the response relevant to the user's message?",
            "helpfulness": "Is the response helpful?",
            "fluency": "Is the response well-written and fluent?",
            "safety": "Is the response safe and appropriate?"
        }

    evaluation_scores = {}
    print("Response Quality Assessment:")
    print("-" * 40)
    print(f"Response: {response_text}")
    print("-" * 40)

    # Evaluate each criterion
    for criterion, description in evaluation_criteria.items():
        # Using simple heuristics for demonstration purposes
        if criterion == "fluency":
            # Base score on sentence structure and length
            score = min(10, max(1, len(response_text.split()) / 5))
        elif criterion == "relevance":
            # Using default score (would need context for better evaluation)
            score = 7
        elif criterion == "helpfulness":
            # Base score on response length (simple heuristic)
            score = min(10, max(1, len(response_text) / 20))
        elif criterion == "safety":
            # Check for potentially problematic terms
            concerning_terms = ["kill", "harm", "illegal", "violent", "dangerous"]
            if any(term in response_text.lower() for term in concerning_terms):
                score = 3
            else:
                score = 9

        # Record and display the score
        evaluation_scores[criterion] = score
        print(f"{criterion.capitalize()} ({description}): {score}/10")

    # Calculate overall score
    average_score = sum(evaluation_scores.values()) / len(evaluation_scores)
    print(f"Overall Score: {average_score:.2f}/10")
    return evaluation_scores

def run_predefined_tests():
    """Test the chatbot with predefined conversation scenarios"""
    test_scenarios = [
        {
            "name": "Initial greeting",
            "messages": [
                {"role": "user", "content": "Hello there!"}
            ]
        },
        {
            "name": "Information request",
            "messages": [
                {"role": "user", "content": "Can you recommend some good movies to watch?"}
            ]
        },
        {
            "name": "Multi-turn conversation",
            "messages": [
                {"role": "user", "content": "I'm planning a vacation."},
                {"role": "assistant", "content": "That sounds exciting! Where are you thinking of going?"},
                {"role": "user", "content": "I'm considering either Italy or Spain."}
            ]
        },
        {
            "name": "Problem solving",
            "messages": [
                {"role": "user", "content": "My smartphone battery drains very quickly. What can I do?"}
            ]
        }
    ]

    print("Running Test Scenarios:")
    for scenario in test_scenarios:
        print("\n" + "=" * 50)
        print(f"Scenario: {scenario['name']}")
        print("=" * 50)

        # Create a fresh conversation for each test
        test_conversation = []

        # Process the scenario messages
        for message in scenario["messages"]:
            test_conversation.append(message)
            print(f"{message['role'].capitalize()}: {message['content']}")

        # Generate response if the last message is from the user
        if test_conversation[-1]["role"] == "user":
            print("\nGenerating response...")
            bot_response = create_bot_response(test_conversation)
            print(f"Assistant: {bot_response}")

            # Evaluate the response quality
            assess_response_quality(bot_response)

    print("\nAll test scenarios completed!")

## Utility Functions and Main Execution

This final cell includes additional utility functions for managing conversations (saving, loading, analyzing) and the main execution block that allows users to select which demo to run. Options include a text-based chat, an interactive widget interface, or automated test scenarios.

In [12]:
# Additional utility functions and main execution

def export_conversation(filename="chatbot_session.txt"):
    """Save the current conversation to a text file"""
    with open(filename, "w") as file:
        file.write(compile_conversation_history(conversation_history))
    print(f"Conversation saved to {filename}")

def import_conversation(filename="chatbot_session.txt"):
    """Load a conversation from a text file"""
    try:
        with open(filename, "r") as file:
            content = file.read()

        # Parse the file content into message format
        parsed_messages = []
        for line in content.split("\n"):
            if line.startswith("User: "):
                parsed_messages.append({"role": "user", "content": line[6:]})
            elif line.startswith("Assistant: "):
                parsed_messages.append({"role": "assistant", "content": line[11:]})

        # Update the conversation history
        conversation_history.clear()
        conversation_history.extend(parsed_messages)
        print(f"Conversation loaded from {filename}")
    except Exception as e:
        print(f"Error loading conversation: {e}")

def show_conversation_analytics():
    """Display statistics and analytics about the conversation"""
    if not conversation_history:
        print("No conversation data to analyze.")
        return

    # Count messages by role
    user_message_count = sum(1 for msg in conversation_history if msg["role"] == "user")
    assistant_message_count = sum(1 for msg in conversation_history if msg["role"] == "assistant")

    # Calculate average message lengths
    user_message_lengths = [len(msg["content"]) for msg in conversation_history if msg["role"] == "user"]
    assistant_message_lengths = [len(msg["content"]) for msg in conversation_history if msg["role"] == "assistant"]

    avg_user_length = sum(user_message_lengths) / len(user_message_lengths) if user_message_lengths else 0
    avg_assistant_length = sum(assistant_message_lengths) / len(assistant_message_lengths) if assistant_message_lengths else 0

    # Display analytics
    print("\n===== Conversation Analytics =====")
    print(f"Total messages: {len(conversation_history)}")
    print(f"User messages: {user_message_count}")
    print(f"Assistant messages: {assistant_message_count}")
    print(f"Average user message length: {avg_user_length:.1f} characters")
    print(f"Average assistant message length: {avg_assistant_length:.1f} characters")

    # Analyze overall sentiment
    all_user_content = " ".join([msg["content"] for msg in conversation_history if msg["role"] == "user"])
    overall_sentiment = detect_message_sentiment(all_user_content)
    print(f"Overall conversation sentiment: {overall_sentiment}")

# Main execution block
if __name__ == "__main__":
    print("\nChatbot System Initialized!")
    print("Select an option:")
    print("1: Start text-based chat")
    print("2: Launch interactive widget interface (best in Colab)")
    print("3: Run test scenarios")
    print("4: Exit")

    user_choice = input("Enter your choice (1-4): ")

    if user_choice == "1":
        start_text_chat()
    elif user_choice == "2":
        launch_widget_interface()
    elif user_choice == "3":
        run_predefined_tests()
    else:
        print("Exiting program. Goodbye!")


Chatbot System Initialized!
Select an option:
1: Start text-based chat
2: Launch interactive widget interface (best in Colab)
3: Run test scenarios
4: Exit
Enter your choice (1-4): 1

===== Conversational Chatbot Demo =====
Type 'exit' to end the conversation

User: Hey How are you?


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Assistant is thinking...
Assistant: Thank you very much.

User: Can you explain me machine learning in simple terms
Assistant is thinking...
Assistant: etc.

User: 4
Assistant is thinking...
Assistant: etc. etc.

User: Exit

Thank you for chatting! Goodbye.
